In [31]:
import pandas
import nltk
from pyspark.sql import SparkSession, functions as F, types as T

In [17]:
spark = (SparkSession
            .builder
            .master("local[*]")
            .appName("Testing")
            .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", 2)
            .config("spark.speculation", False)
            .config("spark.sql.shuffle.partitions", "1")
            .getOrCreate()
        )
    

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 10:30:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [37]:

train_data = spark.read.format('csv').option('header','true').option("escape", "\"").load("data/train")
color_labels_data = spark.read.format('csv').option('header','true').load("data/color_labels.csv")
breed_labels_data = spark.read.format('csv').option('header','true').load("data/breed_labels.csv")
    

In [39]:
pdf = (train_data
           .withColumn("AdoptionSpeed", F.col("AdoptionSpeed").cast(T.IntegerType())).toPandas()
           .merge(breed_labels_data.toPandas(), left_on=["Breed1", "Type"], right_on=["BreedID", "Type"])
           .merge(color_labels_data.toPandas(), left_on="Color1", right_on="ColorID")
           )
pdf["AdoptionSpeed"] = pdf["AdoptionSpeed"].astype("int")
pdf["Fee"] = pdf["Fee"].astype("int")
pdf["Age"] = pdf["Age"].astype("int")
pdf["Name"] = pdf["Name"].fillna("Unknown")
pdf.isna().sum()

Type             0
Name             0
Age              0
Breed1           0
Breed2           0
Gender           0
Color1           0
Color2           0
Color3           0
MaturitySize     0
FurLength        0
Vaccinated       0
Dewormed         0
Sterilized       0
Health           0
Quantity         0
Fee              0
State            0
RescuerID        0
VideoAmt         0
Description      0
PetID            0
PhotoAmt         0
AdoptionSpeed    0
BreedID          0
BreedName        0
ColorID          0
ColorName        0
dtype: int64

In [33]:
import nltk
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/c0b0gyf/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True